# Data Visualization

### imports

In [ ]:
from Audio_segmentation.concat_audio import produce_audio
from Audio_segmentation.split_audio import produce_snippets
from db_connect import db_get_df
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import mutagen.mp3
import os
from transformers import BertModel, BertTokenizer
from tqdm import tqdm

load_dotenv()
audiofile_path = os.getenv("AUDIOFILE_PATH")

In [ ]:
df = db_get_df(table="transcripts_all")
print(len(df))
print(df.dtypes)

Wir verwenden hier das Deutsche model, um die deutschen Transkripte optimal zu encodieren.

### Audio file length

In [ ]:
def get_mp3_lengths(directory):
    mp3_lengths = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".mp3"):
                try:
                    file_path = os.path.join(root, file)
                    audio = mutagen.mp3.MP3(file_path)
                    mp3_lengths.append(audio.info.length)
                except:
                    print(f"file {file} corrupted")
    return mp3_lengths

def plot_mp3_lengths(mp3_lengths):
    plt.hist(mp3_lengths, bins=100, color='cornflowerblue', edgecolor='black')
    plt.xlabel('MP3 Länge (Sekungen)')
    plt.ylabel('Anzahl MP3 Datein')
    plt.title('Länge der MP3 Datein')
    plt.show()

In [ ]:
mp3_directory = audiofile_path
mp3_lengths = get_mp3_lengths(mp3_directory)
plot_mp3_lengths(mp3_lengths)


### Transcript length

In [ ]:
df = db_get_df(table="transcript_word_level_2237")

In [ ]:
filtered_df = df[df['filename'] == 'erich-fried-der-politische-lyriker.mp3']
filtered_df

In [ ]:
def plot_transcript_lengths(transcript_lengths):
    ax = plt.gca()
    ax.set_xlim([1500, 4000])
    plt.hist(transcript_lengths, bins=100, color='cornflowerblue', edgecolor='black')
    plt.xlabel('Transkript Länge (Wörter)')
    plt.ylabel('Anzahl  Transkripte')
    plt.title('Länge der Trankripte')
    plt.show()

In [ ]:
lengths = []
filenames = df["filename"].drop_duplicates(ignore_index=True)
for filename in tqdm(filenames):
    word_entries = df[df['filename'] == filename]
    lengths.append(len(word_entries))

In [ ]:
plot_transcript_lengths(lengths)

### Länge der Sätze

In [ ]:
df = db_get_df(table="transcript_sentences")

In [ ]:
def plot_sentence_lengths(sentence_lengths):
    plt.hist(sentence_lengths, bins=100, color='cornflowerblue', edgecolor='black')
    plt.xlabel('Sätze Länge (Wörter)')
    plt.ylabel('Anzahl  Sätze')
    plt.title('Länge der Sätze')
    plt.show()

In [ ]:
df["sentence_lenght"] = df["sentence"].apply(len)
df = df.sort_values(by="sentence_lenght")

In [ ]:
df.iloc[-1]["sentence"]

In [ ]:
df[df["sentence_lenght"] > 2000]

Logarithmische Normalverteilung

In [ ]:
plot_sentence_lengths(df[df["sentence_lenght"] < 500]["sentence_lenght"])